# 카카오맵 크롤링

search_keyword에 검색할 단어를 입력하고 실행하면 해당 검색 결과에 해당하는 장소들을 크롤링합니다.   
결과는 result 변수.   
(코드 자체는 '카페'정보를 가져오기 위해 작성되었습니다)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep


# 네이버 지도는 접속 시 자동으로 아래 url 뒤에 현재 위치 정보가 입력됨
driver_path = 'C:/Users/namke/jupyter workspace/analysis project/Cafe textmining/driver/chromedriver'
URL = 'https://map.kakao.com/'
search_keyword = '검색어(ex. 서울 카페, 부산 카페 등)'

driver = webdriver.Chrome(executable_path=driver_path)
driver.get(URL)

sleep(5)
driver.find_element_by_class_name('DimmedLayer').click() # 배경에 레이어 하나가 떠서 이걸 지우는 코드

# keyword 검색
search = driver.find_element_by_css_selector('div.box_searchbar>input.query')
search.send_keys(search_keyword)
search.send_keys(Keys.ENTER)

sleep(5)
driver.find_element_by_link_text('장소 더보기').send_keys(Keys.ENTER) # div로 인한 click event 오류로 click 대신 send_keys 사용

sleep(5)
# html 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result = []
i = 1
while True:
    place = soup.select('.PlaceItem')
    for per in place:
        name = per.select_one('.screen_out').text                                          # 장소 이름
        href = per.select_one('.moreview')['href']                                         # 상세보기 주소
        rate = per.select_one('.rating')
        score = rate.select_one('.num').text                                               # 평점
        score_cnt = rate.select_one('.numberofscore').text.replace('건', '')               # 평점 수
        review_cnt = rate.select_one('.review').text.replace('리뷰', '').replace(' ', '')  # 리뷰 수
        addr = per.select_one('.addr').text.replace('\n', '')                              # 장소 주소
        time = per.select_one('.openhour').select_one('a').text                            # 영업 시간
        phone = per.select_one('.phone').text                                              # 전화번호

        # 위 순서대로 저장(column도 동일)
        result.append([name, href, score, score_cnt, review_cnt, addr, time, phone])

    # 페이지 이동
    if i==1 or i%5 != 1:
        try:
            driver.find_element_by_link_text(str(i)).click()
        except:
            break
    else:
        driver.find_element_by_css_selector('#info\.search\.page\.next').click()
    sleep(2)
    i = i + 1

# driver 종료
driver.quit()

검색결과를 dataframe으로 만들어 확인합니다.

In [ ]:
# 크롤링 한 데이터 저장
df = pd.DataFrame(result)
df.columns = ['이름', '링크', '평점', '평점 수 ', '리뷰 수', '주소', '영업시간', '전화번호']
# df.to_csv('result_crawling.csv', index=False, encoding='cp949')
df

결과를 sqlite3 DB에 저장합니다.

In [ ]:
import sqlite3

conn = sqlite3.connect('kakaomap_cafe.db')
df.to_sql('cafe_info_table', conn, chunksize=1000)
conn.commit()
conn.close()

DB에 저장이 잘 되었는지 확인합니다.

In [ ]:
conn = sqlite3.connect('kakaomap_cafe.db')
cur = conn.cursor()
check_table = pd.read_sql('select * from cafe_info_table', conn)
 
conn.commit()
conn.close()

check_table.drop(columns=['index'])